## --- Day 15: Chiton ---
What is the lowest total risk of any path from the top left to the bottom right?

In [1]:
import aoc_utils

class ChitonGraph:
    def __init__(self, filename):
        with open(filename) as f:
            raw = f.read()
        
        self.weights = aoc_utils.str_to_2d_array(raw)
        self.vertices = set()
        self.end_vertex = (len(self.weights)-1, len(self.weights[0])-1)

        for r in range(len(self.weights)):
            for c in range(len(self.weights[0])):
                self.vertices.add((r, c))

    def find_shortest_path(self, verbose=False):
        # Distance of every vertex from the origin (0, 0)
        distance = {v: float("inf") for v in self.vertices}

        # Start at origin, distance=0
        curr_vertex = (0, 0)
        curr_distance = 0

        while curr_vertex != self.end_vertex:
            del(distance[curr_vertex])
            r, c = curr_vertex

            if verbose and len(distance.keys()) % 1000 == 0:
                print(f"{len(distance.keys())} unvisited nodes to go, entering {curr_vertex}")

            # For every adjacent unvisited vertex, update distance from origin
            # (note that not all of these vertices necessarily exist)
            adjacent_vertices = [(r, c-1), (r, c+1), (r-1, c), (r+1, c)]
            for adj_v in adjacent_vertices:
                if adj_v in distance:
                    dist_to_v = self.weight_to(adj_v)
                    distance[adj_v] = min(distance[adj_v], curr_distance + dist_to_v)

            # Get the next vertex with the shortest distance
            min_dist = min(distance.values())
            curr_vertex, curr_distance =\
                next((v, d) for v, d in distance.items() if d == min_dist)
        
        return curr_distance

    def weight_to(self, vertex):
        # Get the weight of any edge leading into vertex
        r, c = vertex

        return self.weights[r][c]

In [2]:
ex1_graph = ChitonGraph("./inputs/Day15ex.txt")
ex1_solution = 40

assert ex1_solution == ex1_graph.find_shortest_path()

In [3]:
p1_graph = ChitonGraph("./inputs/Day15.txt")
p1_graph.find_shortest_path(verbose=True)

9000 unvisited nodes to go, entering (16, 33)
8000 unvisited nodes to go, entering (38, 28)
7000 unvisited nodes to go, entering (47, 34)
6000 unvisited nodes to go, entering (68, 10)
5000 unvisited nodes to go, entering (79, 12)
4000 unvisited nodes to go, entering (64, 53)
3000 unvisited nodes to go, entering (92, 7)
2000 unvisited nodes to go, entering (97, 14)
1000 unvisited nodes to go, entering (47, 99)


390

## Part 2
Using the full map, what is the lowest total risk of any path from the top left to the bottom right?

In [4]:
from heapq import heappush, heappop
import itertools

class VertexQueue:
    """Priority queue to hold vertices and lengths https://docs.python.org/3/library/heapq.html"""
    def __init__(self):
        self.pq = []
        self.entry_finder = {}
        self.counter = itertools.count()

    def add_vertex(self, vertex, distance=float("inf")):
        """Add a vertex to the queue (remove/replace if it is already there)"""
        if vertex in self.entry_finder:
            self.remove_vertex(vertex)
        count = next(self.counter)
        entry = [distance, count, vertex]
        self.entry_finder[vertex] = entry
        heappush(self.pq, entry)

    def remove_vertex(self, vertex):
        """Mark a queue entry as removed by removing it from the entry_finder and setting the vertex to None in the pq"""
        entry = self.entry_finder.pop(vertex)
        entry[-1] = None    # entry is a reference to a list, so this updates the item in the pq

    def pop(self):
        """
        Remove entries from the queue until one references a vertex,
        returns tuple of (vertex, distance)
        """
        while self.pq:
            distance, _, vertex = heappop(self.pq)
            if vertex is not None:
                del(self.entry_finder[vertex])
                return (vertex, distance)
        raise KeyError("Can not pop when queue is empty")

    def contains(self, vertex):
        return vertex in self.entry_finder

    def __getitem__(self, vertex):
        """Return the distance to vertex"""
        return self.entry_finder[vertex][0]

    def __setitem__(self, vertex, distance):
        """Update the vertex distance only if it already exists"""
        if vertex in self.entry_finder:
            self.add_vertex(vertex, distance=distance)
        else:
            raise KeyError

    def __len__(self):
        # Length of the queue is the number of vertices, not the pq length
        return len(self.entry_finder)

In [5]:
class BigChitonGraph(ChitonGraph):
    def __init__(self, filename, scale):
        ChitonGraph.__init__(self, filename)

        # Resize: Add more vertices, update destination
        orig_r, orig_c = self.end_vertex
        for r in range((orig_r+1)*scale):
            for c in range((orig_c+1)*scale):
                self.vertices.add((r, c))

        self.end_vertex = (r, c)

    def weight_to(self, vertex):
        # Translate edge weights as they increase across self.scale # of map tiles
        # in both directions, increasing by 1 with each additional tile to the right
        # and each tile down. Values over 9 wrap back to 1
        r, c = vertex
        tile_size = len(self.weights)
        tile_r = r // tile_size
        tile_c = c // tile_size
        weight = self.weights[r % tile_size][c % tile_size]

        offset_weight = weight + tile_r + tile_c
        if offset_weight > 9:
            offset_weight -= 9

        return offset_weight

    def find_shortest_path(self, verbose=False):
        # Build a queue of every vertex
        vq = VertexQueue()
        for v in self.vertices:
            vq.add_vertex(v)

        # Start by visiting the origin, distance=0
        curr_vertex = (0, 0)
        curr_distance = 0
        vq.remove_vertex(curr_vertex)

        while curr_vertex != self.end_vertex:
            r, c = curr_vertex

            if verbose and len(vq) % 1000 == 0:
                print(f"{len(vq)} unvisited nodes to go, entering {curr_vertex}")

            # For every adjacent unvisited vertex, update distance from origin
            # (note that not all of these vertices necessarily exist)
            adjacent_vertices = [(r, c-1), (r, c+1), (r-1, c), (r+1, c)]
            for adj_v in adjacent_vertices:
                if vq.contains(adj_v):
                    dist_to_v = self.weight_to(adj_v)
                    if curr_distance + dist_to_v < vq[adj_v]:
                        vq[adj_v] = curr_distance + dist_to_v

            # Get the next vertex with the shortest distance
            curr_vertex, curr_distance = vq.pop()
        
        return curr_distance


In [6]:
ex2_graph = BigChitonGraph("./inputs/Day15ex.txt", scale=5)
ex2_solution = 315

assert ex2_solution == ex2_graph.find_shortest_path()

In [7]:
# Part 2 solution
p2_graph = BigChitonGraph("./inputs/Day15.txt", scale=5)
p2_graph.find_shortest_path()

2814